In [ ]:
from pyspark.sql import SparkSession


spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [ ]:
from pyspark import SparkContext
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS
from math import sqrt


data  = spark_session.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferschema", "true")\
        .load("/data/movielens/ratings_100k.csv")\
        .repartition(100)

In [ ]:
ratings = data.rdd.map(lambda l: Rating(int(l[0]),int(l[1]),float(l[2])))

In [ ]:
splits = ratings.randomSplit([1, 1, 1], 0)

train_test = [(sc.union([splits[0], splits[1]]), splits[2]), (sc.union([splits[2], splits[1]]), splits[0]), 
              (sc.union([splits[0], splits[2]]), splits[1])]

In [ ]:
def RMSE(model, data):


    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    ratesAndPreds = predictions.map(lambda x: ((x[0], x[1]), x[2])).join(data.map(lambda x: ((x[0], x[1]), x[2]))).values()
    
    return sqrt(ratesAndPreds.map(lambda x: (x[0] - x[1]) ** 2).mean())

In [ ]:
rank = 20
iterations = 5
lambda_ = 0.01

fold_count = 3

scores = []

for test_fold_index in range(fold_count):
    als = ALS.train(train_test[test_fold_index][0], rank, iterations, lambda_)
    scores.append(RMSE(als, train_test[test_fold_index][1]))

In [ ]:
score = sum(scores) / len(scores)

In [ ]:
print(score)